In [63]:
import pandas as pd
import os

In [64]:
summaryPath =  'data/tripsByMode'
scnJson = '_site/scenarios.json'
outputJsonName = 'zones-modetrips-vizmap.json'
fnSuffix = '_ZoneSummary_TripsByMode.csv'
scenarioDataFolder = '_site/data/scnData'

debug = False

In [65]:
bFillFilterNAWithTextNA = False

# id variables 
lstIdVars = ['TAZID']

# aCode: Code name of attribute. Must begin with lowercase 'a'
# aName: Display name of the attribute
# aRound: how many decimals

# attributes
dfAttributes = pd.DataFrame([
    ['a0','All Trips'          , 4],
    ['aM','Motorized Trips'    , 4],
    ['aA','Auto Trips'         , 4],
    ['aT','Transit Trips'      , 4],
    ['aN','Non-Motorized Trips', 4],
    ['aW','Walk Trips'         , 4],
    ['aB','Bike Trips'         , 4]
], columns=('aCode','aName','aRound'))
display(dfAttributes)


# fCode: Code name of filter. Must begin with lowercase 'f'
# fName: Display name of the filter
# fWidget: what type of widget should the web app use... NOT SURE IF THIS IS USED OR OVERRIDDEN LATER ON
# fOwnCol: True if filter is already a column (long format)
# col: column name in data of filter, only used if fOwnCol==True

dfFilters = pd.DataFrame([
    ["fPurp","Trip Purpose"         , "select"  , True  , 'fPurp'],
    ["fTod" ,"Time of Day"          , "select"  , True  , 'fTod' ],
    ["fPA"  ,"Production/Attraction", "select"  , True  , 'fPA'  ],
    ["fMd"  ,"Mode"                 , "checkbox", False , None   ],
    ["fMd"  ,"Access Mode"          , "checkbox", False , None   ]
], columns=('fCode','fName','fWidget','fOwnCol','col'))

lstFiltersOwnCol = dfFilters[dfFilters['fOwnCol'] == True]['fCode'].tolist()
display(lstFiltersOwnCol)

display(dfFilters)

,aCode,aName,aRound
0,a0,All Trips,4
1,aM,Motorized Trips,4
2,aA,Auto Trips,4
3,aT,Transit Trips,4
4,aN,Non-Motorized Trips,4
5,aW,Walk Trips,4
6,aB,Bike Trips,4


['fPurp', 'fTod', 'fPA']

,fCode,fName,fWidget,fOwnCol,col
0,fPurp,Trip Purpose,select,True,fPurp
1,fTod,Time of Day,select,True,fTod
2,fPA,Production/Attraction,select,True,fPA
3,fMd,Mode,checkbox,False,None
4,fMd,Access Mode,checkbox,False,None


In [66]:
# MAKE SURE ONLY COLUMNS WITH LOWER-CASE "f" ARE FILTER COLUMNS AND LOWER-CASE "a" ARE ATTRIBUTE COLUMNS AND SHOULD MATCH aCode in dfAttributes
dfColKeys = pd.DataFrame([
    ['All'        , None, None, 'a0'],
    ['Moto'       , None, None, 'aM'],
    ['Auto'       , None, None, 'aA'],
    ['dLCL'       ,'LCL',  'D', 'aT'],
    ['dCOR'       ,'COR',  'D', 'aT'],
    ['dEXP'       ,'EXP',  'D', 'aT'],
    ['dLRT'       ,'LRT',  'D', 'aT'],
    ['dCRT'       ,'CRT',  'D', 'aT'],
    ['dBRT'       ,'BRT',  'D', 'aT'],
    ['wLCL'       ,'LCL',  'W', 'aT'],
    ['wCOR'       ,'COR',  'W', 'aT'],
    ['wEXP'       ,'EXP',  'W', 'aT'],
    ['wLRT'       ,'LRT',  'W', 'aT'],
    ['wCRT'       ,'CRT',  'W', 'aT'],
    ['wBRT'       ,'BRT',  'W', 'aT'],
    ['NonM'       , None, None, 'aN'],
    ['Walk'       , None, None, 'aW'],
    ['Bike'       , None, None, 'aB']
], columns=('col' ,'fMd','fMdA','aCode'))

dfColKeys

,col,fMd,fMdA,aCode
0,All,None,None,a0
1,Moto,None,None,aM
2,Auto,None,None,aA
3,dLCL,LCL,D,aT
4,dCOR,COR,D,aT
5,dEXP,EXP,D,aT
6,dLRT,LRT,D,aT
7,dCRT,CRT,D,aT
8,dBRT,BRT,D,aT
9,wLCL,LCL,W,aT


In [67]:
dfScn = pd.read_json(scnJson)
dfScn = dfScn[['modVersion','scnGroup','scnYear']]
display(dfScn)
display(dfScn.dtypes)

,modVersion,scnGroup,scnYear
0,v900,Base,2019
1,v900,Base,2023
2,v900,TIP,2028
3,v900,RTP,2032
4,v900,RTP,2042
5,v900,RTP,2050
6,v900,NoBuild,2032
7,v900,NoBuild,2042
8,v900,NoBuild,2050
9,v900,Needs,2032


modVersion    object
scnGroup      object
scnYear        int64
dtype: object

In [68]:
# create folders
# Define a root directory where you want to create subfolders
root_dir = './_site/data/scnData'

# Ensure the root directory exists
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Loop through the dataframe and create subfolders
for index, row in dfScn.iterrows():
    folder_name = f"{row['modVersion']}__{row['scnGroup']}__{row['scnYear']}"
    full_path = os.path.join(root_dir, folder_name)
    
    if not os.path.exists(full_path):
        os.makedirs(full_path)


In [69]:
import pandas as pd
import os
import glob

# Create an empty list to store dataframes
_dfs = []

# Iterate through the scenarios
for _, row in dfScn.iterrows():
    print(row['scnGroup'] + '__' + str(row['scnYear']))
    
    # Read the CSV into a dataframe
    _df = pd.read_csv(summaryPath +'/' + row['scnGroup'] + '_' + str(row['scnYear']) + "_ZoneSummary_TripsByMode.csv")
    if debug: display(_df)

    # rename filters that are their own columns

    # Create a dictionary for renaming columns
    rename_dict = dfFilters[dfFilters['fOwnCol']==True].set_index('col')['fCode'].dropna().to_dict()

    # Renaming columns in '_df' using the dictionary
    _df.rename(columns=rename_dict, inplace=True)

    _df_melt = _df.melt(id_vars=lstIdVars + lstFiltersOwnCol, value_vars=dfColKeys['col'].tolist(), var_name='col', value_name='val')
    if debug: display(_df_melt)
    _df_melt_withkeys = pd.DataFrame.merge(_df_melt, dfColKeys, on='col')
    if debug: display(_df_melt_withkeys)

    _df_melt_withkeys_pivot_atts = _df_melt_withkeys.pivot(index=[col for col in _df_melt_withkeys.columns if col not in ['aCode','val','col']], columns='aCode', values="val")
    _df_melt_withkeys_pivot_atts.columns.name = None
    _df_melt_withkeys_pivot_atts.reset_index(inplace=True)
    _df_melt_withkeys_pivot_atts
    if debug: display(_df_melt_withkeys_pivot_atts)

    # Add new columns with values from filename
    _df_melt_withkeys_pivot_atts['modVersion'] = row['modVersion']
    _df_melt_withkeys_pivot_atts['scnGroup'] = row['scnGroup']
    _df_melt_withkeys_pivot_atts['scnYear'] = row['scnYear']

    _df_melt_withkeys_pivot_atts['scnYear'] = _df_melt_withkeys_pivot_atts['scnYear'].astype('int64')

    _dfs.append(_df_melt_withkeys_pivot_atts)

# If you want to concatenate all the dataframes into a single dataframe
print('combining dataframes...')
dfCombined_melt_withkeys_pivot_atts = pd.concat(_dfs, ignore_index=True)


# round aCode columns

# Creating a dictionary to map column names to their rounding values
rounding_dict = dict(zip(dfAttributes['aCode'], dfAttributes['aRound']))

# Rounding the columns based on the mapping
for col, round_val in rounding_dict.items():
    if col in dfCombined_melt_withkeys_pivot_atts.columns:
        dfCombined_melt_withkeys_pivot_atts[col] = dfCombined_melt_withkeys_pivot_atts[col].round(round_val)

# fill NA with 'N/A'
if bFillFilterNAWithTextNA:
    dfCombined_melt_withkeys_pivot_atts[[col for col in dfCombined_melt_withkeys_pivot_atts.columns if col.startswith('f')]] = dfCombined_melt_withkeys_pivot_atts[[col for col in dfCombined_melt_withkeys_pivot_atts.columns if col.startswith('f')]].fillna('N/A')

display(dfCombined_melt_withkeys_pivot_atts)

print('Done!')


Base__2019
Base__2023
TIP__2028
RTP__2032
RTP__2042
RTP__2050
NoBuild__2032
NoBuild__2042
NoBuild__2050
Needs__2032
Needs__2042
Needs__2050
Needs MAG__2050
combining dataframes...


,TAZID,fPurp,fTod,fPA,fMd,fMdA,a0,aA,aB,aM,aN,aT,aW,modVersion,scnGroup,scnYear
0,2,All,Dy,a,NaN,NaN,38.8419,33.0394,0.3943,33.0394,5.8025,NaN,5.4082,v900,Base,2019
1,2,All,Dy,a,BRT,D,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Base,2019
2,2,All,Dy,a,BRT,W,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Base,2019
3,2,All,Dy,a,COR,D,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Base,2019
4,2,All,Dy,a,COR,W,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Base,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18689185,3629,HBW,Pk,p,EXP,W,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Needs MAG,2050
18689186,3629,HBW,Pk,p,LCL,D,NaN,NaN,NaN,NaN,NaN,0.0069,NaN,v900,Needs MAG,2050
18689187,3629,HBW,Pk,p,LCL,W,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Needs MAG,2050
18689188,3629,HBW,Pk,p,LRT,D,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,v900,Needs MAG,2050


Done!


In [70]:
## replace zeros with NaN
#import numpy as np
#dfCombined_melt_withkeys_pivot_atts.replace(0, np.nan, inplace=True)
#if debug: display(dfCombined_melt_withkeys_pivot_atts)

In [74]:
import json

# PREPARE JSON FOR EACH SCENARIO
# scenarioDetails
# attributes
#   aCode
#   aName
#   filterGroup
# filterGroups
#   possible filterGroup Options
#   for each filterGroup possibility, the data for that selection

_df0 = dfCombined_melt_withkeys_pivot_atts.copy()

# Loop through scenarios in dfScn
for _, row in dfScn.iterrows():

    _scenarioCode = row['modVersion'] + '__' + row['scnGroup'] + '__' + str(row['scnYear'])
    print(_scenarioCode)

    # format scenario details for json export
    jsonScenarioDetails = row.to_dict()

    #print(row['modVersion'] + row['scnGroup'] + str(row['scnYear']))
    # Filter the main dataframe based on the scenario
    _df0_filtered = _df0[
        (_df0['modVersion'] == row['modVersion']) & 
        (_df0['scnGroup'] == row['scnGroup']) & 
        (_df0['scnYear'] == row['scnYear'])
    ].copy()

    _df = _df0_filtered.copy()
    if debug: display(_df)

    # add options to _dfFiltersWithOptions
    
    _dfFiltersWithOptions = dfFilters.copy()

    # For each fCode, get the unique values from dfData, and join to dfFilters
    formatted_lists = []
    for fCode in dfFilters['fCode']:
        unique_values = [x for x in _df[fCode].unique() if pd.notna(x)]
        formatted_lists.append(unique_values)
    if debug: display (formatted_lists)

    _dfFiltersWithOptions['fOptions'] = formatted_lists    
    if debug: display(_dfFiltersWithOptions)

    # get list of fitler groups (any column that begins with f)
    fCols = [col for col in dfCombined_melt_withkeys_pivot_atts.columns if col.startswith('f')]
    _df[fCols] = _df[fCols].fillna('')
    _dfFilters = _df[fCols].drop_duplicates()

    # Drop rows with empty strings in ALL columns
    #_dfFilters = _dfFilters[~(_dfFilters == '').all(axis=1)]
    if debug: display(_dfFilters)

    # Convert the values to binary form, so 1 where there is a value and 0 elsewhere
    binary_dfFilters = _dfFilters.applymap(lambda x: 0 if x=='' else 1)
    if debug: display(binary_dfFilters)

    # Get unique rows of combinations
    unique_combinations = binary_dfFilters.drop_duplicates()

    # Convert the unique combinations to a list of tuples
    lstCombinations = list(unique_combinations.itertuples(index=False, name=None))
    if debug: display(lstCombinations)

    # Define a function to convert to binary
    def is_empty_or_not(value):
        return 0 if value=='' else 1

    _dfFilterACodes = pd.DataFrame()

    jsonDataCombined = {}

    # loop through all combinations in list
    for combination in lstCombinations:

        # get list of column names with a 1 for given combination
        _dfFilterColsWithValue1 = [col for idx, col in enumerate(fCols) if combination[idx] == 1]
        if debug: display(_dfFilterColsWithValue1)

        # create string of column names
        _strFilterGroup = '_'.join(_dfFilterColsWithValue1)
        if debug: display(_strFilterGroup)
        
        # Initialize the condition as True for all rows
        condition = pd.Series([True] * len(_df), index=_df.index)

        # Apply the condition programmatically for each column in the list
        for col, comb_value in zip(fCols, combination):
            condition &= _df[col].apply(is_empty_or_not) == comb_value

        _filtered_df = _df[condition].copy()
        _filtered_df.drop(columns=['modVersion','scnGroup','scnYear'], inplace=True)
        if debug: display(_filtered_df)

        # Remove columns where all values are NaN
        _filtered_df_noNA = _filtered_df.dropna(axis=1, how='all')
        if debug: display(_filtered_df_noNA)

        _fColsToDrop = [col for col in _filtered_df_noNA.columns if col.startswith('f') and (_filtered_df_noNA[col] == '').all()]

        # Drop the identified columns - only have fCols with values
        _df2 = _filtered_df_noNA.drop(columns=_fColsToDrop)
        if debug: display(_df2)

        # Get list of columns that begin with "a", create a dataframe the filter group for all possible combinations
        aCols = [col for col in _filtered_df_noNA.columns if col.startswith('a')]
        new_data = pd.DataFrame([{"filterGroup": _strFilterGroup, 'aCode': aCols}])
        new_data = new_data.explode('aCode').reset_index(drop=True)
        _dfFilterACodes = pd.concat([_dfFilterACodes, new_data], ignore_index=True)

        # get unique filter values
        _fColsRemain = [col for col in _df2.columns if col.startswith('f')]
        if debug: display(_fColsRemain)
        if _fColsRemain:
            _dfFilterOptions = _df2[_fColsRemain].drop_duplicates()
        else:
            # add dummy row for no filter
            _dfFilterOptions = pd.DataFrame({'nofilter': ['']})

        if debug: display(_dfFilterOptions)

        # Handle no filter data

        first_row = True

        for index, row in _dfFilterOptions.iterrows():

            # Function to convert a value to an integer if it's a whole number
            def convert_to_int_if_whole_number(value):
                try:
                    float_val = float(value)
                    if float_val.is_integer():
                        return str(int(float_val))
                    else:
                        return str(float_val)
                except ValueError:
                    return str(value)

            # Applying the function to each element and joining with '_'
            _filterRowIndex = '_'.join(row.apply(convert_to_int_if_whole_number).values)

            #print(_filterRowIndex)
            
            final_condition = ""

            if _fColsRemain != []:
                # Dynamically construct the filter condition
                shared_columns = set(_dfFilterOptions.columns) & set(_df2.columns)
                conditions = (_df2[col].isin([row[col]]) for col in shared_columns)
                final_condition = next(conditions)

                for condition in conditions:
                    final_condition &= condition

                filtered_df2 = _df2[final_condition].copy()
                filtered_df2.drop(columns=_fColsRemain, inplace=True)
            else: # no filter condition
                filtered_df2 =  _df2

            _df2noI = filtered_df2.reset_index()

            jsonDataForFilterOption = {}

            for index, row in _df2noI.iterrows():
                # Initialize an empty dictionary for the row
                row_dict = {}

                # Loop through each column that starts with 'a'
                for col in _df2noI.columns[_df2noI.columns.str.startswith('a')]:
                    # Add to row_dict if value is greater than 0
                    if row[col] > 0:
                        row_dict[col] = row[col]
                
                # If row_dict is not empty, add it to jsonDataCombined with TAZID as the key
                if row_dict:
                    jsonDataForFilterOption[int(row['TAZID'])] = row_dict 
            
            # Encapsulate jsonDataCombined under the _filterRowIndex
            final_json = {_filterRowIndex: jsonDataForFilterOption}

            ## Convert to a JSON formatted string
            #json_output = json.dumps(final_json)

            # Append the JSON structure from this iteration to the combined
            jsonDataCombined = {**jsonDataCombined, **final_json}

    _dfAttributesWFilter = pd.DataFrame.merge(dfAttributes, _dfFilterACodes, on='aCode')
    jsonAttributes = _dfAttributesWFilter.to_dict(orient='records')
    jsonFilters = _dfFiltersWithOptions.to_dict(orient='records')

    # Construct final JSON structure
    jsonScenario = {
        'scenarioDetails': jsonScenarioDetails,
        'attributes': jsonAttributes,
        'filters': jsonFilters,
        'data': jsonDataCombined#,
        #'aggregations': aggregations_list
    }

    # Specify the desired file name
    filename = '_site/data/scnData/' + _scenarioCode + '/' + outputJsonName

    # Open the file in write mode
    with open(filename, 'w') as file:
        json.dump(jsonScenario, file)
        
    if debug: break

display('Done!')

v900__Base__2019
v900__Base__2023
v900__TIP__2028
v900__RTP__2032
v900__RTP__2042
v900__RTP__2050
v900__NoBuild__2032
v900__NoBuild__2042
v900__NoBuild__2050
v900__Needs__2032
v900__Needs__2042
v900__Needs__2050
v900__Needs MAG__2050


'Done!'